**Created on 05/01/2023**

**Authors: Thomas ROSTAINGT**

**License: this code is released under the CeCILL 2.1 license. See https://www.cecill.info/licenses/Licence_CeCILL_V2.1-en.txt**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

### Importation des données

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/M2 SID/PIP2023/data_v2.csv')

In [4]:
df.head()

,Unnamed: 0,initiator,target,protocol,nlos_indicator,t1,t2,t3,t4,skew,...,temperature_initiator,temperature_target,timestamp,seqnum,rssi_request,rssi_ack,rssi_data,rssi_mean,register_dump_CIR,orientation
0,0,183,100,TWR,2.783982,946218857614,946490292658,946594234510,946322802711,-2.368804,...,30.98,30.220001,1654929794228,4,-80.136574,-80.077130,-79.836296,-80.016667,"{""ACC_MEM"": ""AAT/9gACAAr/+wABAAL/8wAS//AACgAGA...",270.0
1,1,183,100,TWR,2.920485,1009474709646,1009746235572,1009850439822,1009578917162,-1.045965,...,30.98,29.840002,1654929795219,10,-80.267769,-79.988684,-79.835720,-80.030724,"{""ACC_MEM"": ""//gAEwAC//X/9gAXABEACwASABQAFgAG/...",270.0
2,2,183,100,TWR,2.536090,1072923068046,1073194681670,1073299179662,1073027569304,-0.799856,...,30.98,29.840002,1654929796210,16,-79.964310,-79.850138,-79.693502,-79.835983,"{""ACC_MEM"": ""//QABQAJ//X/9gAUABMADv/3//r//gAK/...",270.0
3,3,183,100,TWR,2.857669,36731534990,37003232943,37107270798,36835576116,-0.984438,...,30.98,29.840002,1654929797201,22,-80.040833,-79.818313,-79.849525,-79.902890,"{""ACC_MEM"": ""ACb//AAa//4ADQAJAAYALf/+ABQAEgAOA...",270.0
4,4,183,100,TWR,2.263103,99923195534,100194974620,100298828942,100027053147,-0.769092,...,30.98,29.840002,1654929798190,28,-79.999908,-80.102552,-80.022316,-80.041592,"{""ACC_MEM"": ""//b/+P/6//3/8QAI//H//P/1AAkAAv/3A...",270.0


In [5]:
df.columns

Index(['Unnamed: 0', 'initiator', 'target', 'protocol', 'nlos_indicator', 't1',
       't2', 't3', 't4', 'skew', 'tof', 'tof_skew', 'range', 'range_skew',
       'ranging_unit', 'ranging_error', 'loc_initiator_x', 'loc_initiator_y',
       'loc_initiator_z', 'loc_target_x', 'loc_target_y', 'loc_target_z',
       'distance', 'course_dist', 'temperature_initiator',
       'temperature_target', 'timestamp', 'seqnum', 'rssi_request', 'rssi_ack',
       'rssi_data', 'rssi_mean', 'register_dump_CIR', 'orientation'],
      dtype='object')

# Sélection de variables

In [6]:
# sélection de variables version Jingmeng
col = df.columns
col = ['nlos_indicator', 't1', 't2', 't3','t4', 'skew', 'tof', 'tof_skew', 'loc_target_x', 'loc_target_y', 'rssi_request', 'rssi_ack','rssi_data','rssi_mean']
X = df[col]
y = df[['loc_initiator_x']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# sélection de variables version Abdou
col = ['nlos_indicator','t1','skew','tof','range','loc_target_z','distance','course_dist','temperature_initiator','timestamp','seqnum','rssi_mean']
df[col] = df[col].astype(float)
X = df[col]
y = df[['loc_initiator_x']]

générer deux nouvelles colonnes :
- T3-T2
- T4-T1

### Création jeu d'entraînement et de test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

##XGboost

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

# Mesures de performance
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [ ]:
type(y_train)

pandas.core.frame.DataFrame

In [ ]:
for col in X_train.columns:
  print(col,':',type(col))
# toutes les colonnes sont au format string

nlos_indicator : <class 'str'>
t1 : <class 'str'>
skew : <class 'str'>
tof : <class 'str'>
range : <class 'str'>
loc_target_z : <class 'str'>
distance : <class 'str'>
course_dist : <class 'str'>
temperature_initiator : <class 'str'>
timestamp : <class 'str'>
seqnum : <class 'str'>
rssi_mean : <class 'str'>


In [ ]:
# Méthode de sélection de paramètres (expérimentale, à comparer avec le gridsearch traditionnel) 
#----------------------------------------------------------------------------------------------#
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
import pandas as pd

param_grid = {'n_estimators':[50,100,200], # [50, 70, 80, 90] # Nombre d'arbres, plus il y a d'arbres plus l'algo a tendance à surapprendre sur les données
              'loss':['squared_error', 'absolute_error', 'huber', 'quantile'], # 
              'learning_rate':[0.01,0.1], # taux d'apprentissage => permet de limiter le surapprentissage
              'criterion':['friedman_mse', 'squared_error','mse'], #
              'max_features':['auto', 'sqrt', 'log2']}

model = GradientBoostingRegressor()
sh = GridSearchCV(model, param_grid, cv=5).fit(X_train, y_train)
sh.best_estimator_ # renvoie la meilleure combinaison de paramètres du modèle

In [ ]:
# choix du meilleur paramètre
param_grid = {'n_estimators':[50,100,200], # [50, 70, 80, 90] # Nombre d'arbres, plus il y a d'arbres plus l'algo a tendance à surapprendre sur les données
              'loss':['squared_error', 'absolute_error', 'huber', 'quantile'], # 
              'learning_rate':[0.01,0.1], # taux d'apprentissage => permet de limiter le surapprentissage
              'criterion':['friedman_mse', 'squared_error','mse'], #
              'max_features':['auto', 'sqrt', 'log2']}

# 
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Affiche le détails pour les autres paramètres
moy = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(moy, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(),
             param_grid={'criterion': ['friedman_mse', 'squared_error', 'mse'],
                         'learning_rate': [0.01, 0.1],
                         'loss': ['squared_error', 'absolute_error', 'huber',
                                  'quantile'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [50, 100, 200]})

# Test du modèle XGBoost selon les différents scénarios

**Scénario 1** \\
Jeu de train : composé à 75% du dataset initial \\
Jeu de test : composé à 25% du dataset initial \\

**Scénario 2**
Jeu de train : composé de tous les capteurs du dataset initial sauf un \\
Jeu de test : composé du capteur mis de côté du dataset initial \\

**Scénario 3** \\
Jeu de train : dataset initial \\
Jeu de test : dataset généré avec les rotations

**Scénario 4**  \\
Jeu de train : dataset initial sans le capteur 150 \\
Jeu de test : 

**Scénario 5** \\
Jeu de train : dataset avec 75% des signaux du capteur 150 \\
Jeu de test : dataset avec 25% des signaux du capteur 150




## Scénario 1

In [9]:
# séparation des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
# définition du modèle
xgb_reg = GradientBoostingRegressor(n_estimators = 150, criterion = 'friedman_mse', learning_rate = 0.1, loss = 'squared_error', max_features = 'auto', max_depth = 4) # initialisation du modèle
# entraînement du modèle
xgb_reg.fit(X_train, y_train)
# prédiction du modèle sur le jeu de test
predictions = xgb_reg.predict(X_test)
predictions = pd.DataFrame(predictions, columns=['Predictions'])
# calcul des métriques
print('mse : ',mean_squared_error(y_test, predictions, squared=False))
print('r2 : ',r2_score(y_test, predictions))

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


mse :  0.38879308669092283
r2 :  0.8408636541587897


## Scénario 2 

In [ ]:
col = ['nlos_indicator', 't1', 't2', 't3','t4', 'skew', 'tof', 'tof_skew', 'loc_target_x', 'loc_target_y', 'rssi_request', 'rssi_ack','rssi_data','rssi_mean']

In [ ]:
df_train = df[df.target!=150][col]

In [ ]:
# séparation des données
df_train = df[df.target!=150]
df_test = df[df.target==150]

y_train = df_train['loc_initiator_x']
X_train = df_train[col]

y_test = df_test['loc_initiator_x']
X_test = df_test[col]

In [ ]:
# définition du modèle
xgb_reg = GradientBoostingRegressor(n_estimators = 150, criterion = 'friedman_mse', learning_rate = 0.1, loss = 'squared_error', max_features = 'auto', max_depth = 4) # initialisation du modèle
# entraînement du modèle
xgb_reg.fit(X_train, y_train)
# prédiction du modèle sur le jeu de test
predictions = xgb_reg.predict(X_test)
predictions = pd.DataFrame(predictions, columns=['Predictions'])
# calcul des métriques
print('mse : ',mean_squared_error(y_test, predictions, squared=False))
print('r2 : ',r2_score(y_test, predictions))

mse :  2.220290421521533
r2 :  -8.634760057572766


In [ ]:
xgb_reg.score(X_test, y_test)

-8.634760057572766

## Scénario 3

## Scénario 4